In [1]:
#!python -m pip install pymongo
#!pip install python-dotenv
#!pip install bs4

In [2]:
from pymongo import MongoClient
from bson import ObjectId
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import unicodedata
import os
from dotenv import load_dotenv
from utils import espiadinha, get_df
load_dotenv()

True

<h3>Funções para acessar MongoDB Alexandria:  </h3>
*Utilize "help(nome da função)" para entender os atributos da função (exemplo: help(espiadinha))

In [3]:
#credenciais de acesso ao alexandria
server_type = os.getenv("server_type")
MONGO_ALEXANDRIA_USERNAME = os.getenv('MONGO_ALEXANDRIA_USERNAME')
MONGO_ALEXANDRIA_PASSWORD = os.getenv('MONGO_ALEXANDRIA_PASSWORD')
MONGO_ALEXANDRIA_HOST = os.getenv('MONGO_ALEXANDRIA_HOST')
str_conn = f"{server_type}://{MONGO_ALEXANDRIA_USERNAME}:{MONGO_ALEXANDRIA_PASSWORD}@{MONGO_ALEXANDRIA_HOST}" 
client = MongoClient(str_conn)
db = client['alexandria-kroton']

Primeiro: para ter acesso a todas as collections (Bases) do Mongo

In [4]:
#for i  in db.list_collection_names():
#    print(i)

Para acessar essas bases use as funções espiadinha e get_df:
As duas transformam a base em dataframe pandas

In [5]:
#espiadinha(df,'disciplines',1)
#df_disc=get_df(db,'disciplines',['title','learningUnits'])

Instanciando bases necessárias para o projeto

In [6]:
#import das bases do alexandria
cursos={
        'Pedagogia':['PRÁTICAS PEDAGÓGICAS: IDENTIDADE DOCENTE​','PENSAMENTO CIENTÍFICO','FUNCIONAMENTO DA EDUCAÇÃO BRASILEIRA E POLÍTICAS PÚBLICAS​','FUNDAMENTOS DA EDUCAÇÃO​','PSICOLOGIA DA EDUCAÇÃO E DA APRENDIZAGEM'],
        'Administração':['MATEMÁTICA FINANCEIRA','LEGISLAÇÃO EMPRESARIAL APLICADA','ANÁLISE DE CUSTOS​','MÉTODOS QUANTITATIVOS​','OPTATIVA I (EDUCAÇÃO AMBIENTAL;GESTÃO DO CONHECIMENTO)​'],
        'Gestão em RH':['MATEMÁTICA FINANCEIRA','LEGISLAÇÃO EMPRESARIAL APLICADA','GESTÃO DE PESSOAS​','MÉTODOS QUANTITATIVOS​','OPTATIVA  II (AVALIAÇÃO DE PERFORMANCE; COMUNICAÇÃO E EDUCAÇÃO CORPORATIVA; LIBRAS - LÍNGUA BRASILEIRA DE SINAIS)​','PROJETO INTEGRADO INOVAÇÃO - GESTÃO​'],
        'Desenvolvimento de Sistemas':['ENGENHARIA DE SOFTWARE​','LINGUAGEM DE PROGRAMAÇÃO​','LÓGICA E MATEMÁTICA COMPUTACIONAL','ALGORITMOS E PROGRAMAÇÃO ESTRUTURADA​','ANÁLISE E MODELAGEM DE SISTEMAS​','PROJETO INTEGRADO INOVAÇÃO - ANÁLISE E DESENVOLVIMENTO DE SISTEMAS​'],
        'Serviço Social':['INTRODUÇÃO À FILOSOFIA​','ESTATÍSTICA E INDICADORES SOCIAIS​','ADMINISTRAÇÃO E PLANEJAMENTO DE SERVIÇO SOCIAL​','LEGISLAÇÃO SOCIAL E DIREITOS HUMANOS​','ECONOMIA POLÍTICA​'],
        'Perícia Criminal':['FUNDAMENTOS HISTÓRICOS E INTRODUÇÃO AO ESTUDO DO DIREITO','FUNDAMENTOS DE INVESTIGAÇÃO E CRIMINALÍSTICA​','TEORIA GERAL DO PROCESSO​','EXPANSÃO DA CRIMINALIDADE​','TEORIA GERAL DO DIREITO CONSTITUCIONAL','PROJETO INTEGRADO INOVAÇÃO - INVESTIGAÇÃO E PERÍCIA CRIMINAL​'],
    }

df_disc=get_df(db,'disciplines',['title','learningUnits','krotonId'])
df_LUs=get_df(db,'learningunits',['_id','classes'])
df_cls=get_df(db,'classes',['_id','blocks','krotonId','title'])
df_blk=get_df(db,'blocks',['_id','questions','content','title'])
df_quest=get_df(db,'questions',['_id','type','questionModel','title','description','feedback','alternatives','classId'])

----

<h1>Esqueleto de Projeto</h1>

<h3>Requisições de questões de prova</h3>

In [7]:
#Exploração via Pandas do conteúdo de quiz e prova a partir de uma matéria
materia = 'Administração'
numero_curso = 1
filtro_materia = cursos[materia][numero_curso]
filtro_materia = ''.join(c for c in filtro_materia if not unicodedata.category(c).startswith('C'))
print(f'Matéria: {materia}')
print('Curso: ',filtro_materia)

#Filtro na base de disciplinas para pegar as Learning Units (Semanas de aula) relacionadas a tais disciplinas
adm_LUs = df_disc[df_disc.title.isin([filtro_materia])]['learningUnits'].to_list()[-1]

print("\nEssa é a base de disciplinas. A coluna learningUnits possui uma lista com diversos IDs, cada um se refere à uma semana de aula.\n Estamos pegando a ultima semana para encontrar exercícios de prova")
df_disc[df_disc.title.isin([filtro_materia])]

Matéria: Administração
Curso:  LEGISLAÇÃO EMPRESARIAL APLICADA

Essa é a base de disciplinas. A coluna learningUnits possui uma lista com diversos IDs, cada um se refere à uma semana de aula.
 Estamos pegando a ultima semana para encontrar exercícios de prova


,title,learningUnits,krotonId
262,LEGISLAÇÃO EMPRESARIAL APLICADA,"[658c34fda938399447e4e153, 658c34fda938399447e...",ce0a0fdc-8275-46c2-a2bd-bbd0d5ed4653


In [8]:
#Filtro sobre a base de Learning Units para retornar as respectivas Classes (Aulas)
adm_CLASSES = df_LUs[df_LUs._id.isin(adm_LUs)]['classes'].to_list()[-1]

print("\nEssa é a base de learningUnits. Cada linha é uma semana de aula. A coluna classes possui uma lista com diversos IDs, cada um se refere à uma aula da semana.\n Estamos pegando a ultima aula para encontrar exercícios de prova")
df_LUs[df_LUs._id.isin(adm_LUs)]


Essa é a base de learningUnits. Cada linha é uma semana de aula. A coluna classes possui uma lista com diversos IDs, cada um se refere à uma aula da semana.
 Estamos pegando a ultima aula para encontrar exercícios de prova


,_id,classes
905,658c34fda938399447e4e153,"[658c3586cb94b24a5fe56be7, 658c3587a938399447e..."
906,658c34fda938399447e4e154,"[658c36b5cb94b24a5fe56f38, 658c36b6cb94b24a5fe..."
907,658c34fda938399447e4e156,"[658c3618cb94b24a5fe56d4f, 658c3618cb94b24a5fe..."
908,658c34fda938399447e4e155,"[658c564ca938399447faab89, 658c564dcb94b24a5ff..."


In [9]:
#Filtro sobre a base de classes para retornar os respectivos blocos de aula (Assets: conteúdos que serão disponibilizados ao aluno no AVA)
adm_classes_blocks = df_cls[df_cls._id.isin(adm_CLASSES)]['blocks'].to_list()[-1]

print("\nEssa é a base de Blocos. Cada linha é um Asset de aula.\n A coluna content possui o conteúdo da aula.\n Já a coluna questions possui Ids com questões. Devemos pegar a linha que o título seja Questões de prova.")
df_blk[df_blk._id.isin(adm_classes_blocks)]


Essa é a base de Blocos. Cada linha é um Asset de aula.
 A coluna content possui o conteúdo da aula.
 Já a coluna questions possui Ids com questões. Devemos pegar a linha que o título seja Questões de prova.


,_id,questions,content,title
37752,66ad05d369b42deaa621d4dc,[],"<p>Estudante, esta videoaula foi preparada esp...",Videoaula de Encerramento
37753,66ad05d369b42deaa621d4dd,[],"<p><span style=""color:windowtext;"">Olá, estuda...",Ponto de Chegada
37754,66ad05d369b42deaa621d4de,[],,É Hora de Praticar!
37755,66ad05d369b42deaa621d4df,"[66ad0a0a962d1d1ede949e52, 66ad0a4d962d1d1ede9...",None,Questões Modelo ENADE - objetivas somativas
37756,66ad05d369b42deaa621d4e0,"[66ad07af962d1d1ede911cec, 66ad07f7962d1d1ede9...",None,Questões de Fim de Unidade - Objetivas somativas
37757,66ad05d369b42deaa621d4e1,"[66ad0acc962d1d1ede95cf8f, 66ad0b1069b42deaa62...",None,Questões de Prova Presencial - objetivas somat...
37758,66ad05d369b42deaa621d4e2,[66ad0bc069b42deaa627c42d],None,Questão discursiva


In [10]:
# Cria lista de IDs de questões da disciplina
filtro_questoes = df_blk[df_blk._id.isin(adm_classes_blocks)&(df_blk.title.str.contains('Questões de Prova'))]['questions'].to_list()[0]
questoes_prova = df_quest[df_quest._id.isin(filtro_questoes)]
print("\nEssa é a base de questões filtrada com apenas questões de prova.")
questoes_prova


Essa é a base de questões filtrada com apenas questões de prova.


,_id,type,questionModel,title,description,feedback,alternatives,classId
23280,66ad0acc962d1d1ede95cf8f,multiple-choice,classroom,<p>A relação jurídica de consumo envolve dois ...,<p>Assinale a alternativa que apresenta a cara...,<p>A característica marcante do consumidor na ...,"[{'text': '<p>Vulnerabilidade técnica.</p>', '...",658c564ca938399447faab89
23281,66ad0b1069b42deaa627c260,multiple-choice,classroom,<p>O acesso à justiça é fundamental para a efe...,<p>Assinale a alternativa que apresenta o prop...,<p>A facilitação da defesa dos direitos do con...,[{'text': '<p>Restringir o acesso do consumido...,658c564dcb94b24a5ffa3eaa
23282,66ad0b63962d1d1ede95cfed,multiple-choice,classroom,<p>As práticas abusivas contratuais no context...,<p>Assinale a alternativa que é considerada ab...,"<p>O artigo 51, I, do CDC considera nulas de p...",[{'text': '<p>Cláusulas que limitam a responsa...,658c564d84b23177f6421ec8
23283,66ad0b9e69b42deaa627c3a3,complex-multiple-choice,classroom,<p>A responsabilidade civil nas relações consu...,<p>Com base na caracterização da responsabilid...,"<p>Na responsabilidade subjetiva, o elemento c...",[{'text': '<p>A culpa é desnecessária para a r...,658c564ecb94b24a5ffa3eb3


----

In [11]:
for n in range(questoes_prova.shape[0]):
        print('\n',('-'*100))
        print('questão ',n+1, ' - tipo: ',questoes_prova.iloc[n]['type'])
        print('Título: ', questoes_prova.iloc[n]['title'])
        print('Comando: ', questoes_prova.iloc[n]['description'])
        print('Alternativas: ', questoes_prova.iloc[n]['alternatives'])
        print('Resposta Prof: ', questoes_prova.iloc[n]['feedback'])
        print('\n')


 ----------------------------------------------------------------------------------------------------
questão  1  - tipo:  multiple-choice
Título:  <p>A relação jurídica de consumo envolve dois sujeitos fundamentais: o consumidor e o fornecedor. Ambos desempenham papéis distintos e têm direitos e deveres definidos pelo Código de Defesa do Consumidor (CDC).</p>
Comando:  <p>Assinale a alternativa que apresenta a característica marcante do consumidor na relação jurídica de consumo:&nbsp;</p>
Alternativas:  [{'text': '<p>Vulnerabilidade técnica.</p>', 'isRightAnswer': True, '_id': ObjectId('66ad0acc962d1d1ede95cf90')}, {'text': '<p>Conhecimento específico.</p>', 'isRightAnswer': False, '_id': ObjectId('66ad0acc962d1d1ede95cf91')}, {'text': '<p>Capacidade econômica superior.</p>', 'isRightAnswer': False, '_id': ObjectId('66ad0acc962d1d1ede95cf92')}, {'text': '<p>Participação no processo produtivo.</p>', 'isRightAnswer': False, '_id': ObjectId('66ad0acc962d1d1ede95cf93')}, {'text': '<p>Int

----
<h3>Requisições de conteúdo de aula</h3>

In [12]:
print('Lambrando que já temos os IDs das learning Units')
df_LUs[df_LUs._id.isin(adm_LUs)]

Lambrando que já temos os IDs das learning Units


,_id,classes
905,658c34fda938399447e4e153,"[658c3586cb94b24a5fe56be7, 658c3587a938399447e..."
906,658c34fda938399447e4e154,"[658c36b5cb94b24a5fe56f38, 658c36b6cb94b24a5fe..."
907,658c34fda938399447e4e156,"[658c3618cb94b24a5fe56d4f, 658c3618cb94b24a5fe..."
908,658c34fda938399447e4e155,"[658c564ca938399447faab89, 658c564dcb94b24a5ff..."


In [13]:
#Seleção de todas as aulas da disciplina

ids_classes = []
for i in df_LUs[df_LUs._id.isin(adm_LUs)]['classes']:
    print(len(i))
    ids_classes.extend(i)
print(len(ids_classes))

df_cls[df_cls._id.isin(ids_classes)]

5
5
5
5
20


,_id,blocks,krotonId,title
4332,658c3586cb94b24a5fe56be7,"[6668a2199bab0fa8a141b0e5, 6668a2199bab0fa8a14...",8af28eb9-b93b-4c59-8c46-666e4deb3481,Teoria Geral Da Empresa
4333,658c3587a938399447e5c1fa,"[6668a83f9bab0fa8a141c788, 6668a83f9bab0fa8a14...",358437c0-f3cc-4768-890b-dfaf16f65c22,Teoria Geral Do Direito Societário
4334,658c3588cb94b24a5fe56bf0,"[66698ba21a3421cc9de114a7, 66698ba21a3421cc9de...",df6f2eb8-8937-4e1c-89d8-6061f39e42ed,Falência E Recuperação Judicial Da Empresa
4335,658c3588cb94b24a5fe56bf9,"[666991e91a3421cc9de20ba6, 666991e91a3421cc9de...",66ed2733-1675-47ff-ba6c-b2ba69cf6734,Propriedade Industrial
4336,658c3618cb94b24a5fe56d4f,"[66ad12e4962d1d1edea5128e, 66ad12e4962d1d1edea...",8b9a08ed-b845-4915-ba0f-ec99526d4479,Relação De Emprego E Contrato De Trabalho
4337,658c3618cb94b24a5fe56d58,"[66ad1527962d1d1edea5189d, 66ad1527962d1d1edea...",87783b4e-12fc-4ff2-b4fa-860ca486aa5d,Jornada De Trabalho E Direito Ao Descanso
4338,658c361984b23177f62882da,"[66ad173d962d1d1edea64b0b, 66ad173d962d1d1edea...",a7f70cbc-c302-4d4d-99f4-a976f807552b,Remuneração e Salário
4339,658c361a84b23177f62882e3,"[66ad1b47962d1d1edea77d89, 66ad1b47962d1d1edea...",5c694fc6-a6e9-4427-a48d-ec1ca4b0cb73,Término do Contrato de Trabalho
4340,658c36b5cb94b24a5fe56f38,"[66acbd5b962d1d1ede669d22, 66acbd5b962d1d1ede6...",d352bd1b-ce67-4efe-92c1-b4dcddac0d40,Teoria Geral do Tributo
4341,658c36b6cb94b24a5fe56f41,"[66acbfdd962d1d1ede669ffd, 66acbfdd962d1d1ede6...",7198a500-d8de-48c0-9104-364dd7e11e6b,Obrigação e Responsabilidade Tributária


In [25]:
conteudo_str=''
for n,i in enumerate(df_cls[df_cls._id.isin(ids_classes)]['blocks'].to_list()):
    conteudo_str= conteudo_str + f'\n==>> Aula {n}: '
    for texto in df_blk[df_blk._id.isin(i)][['title','content']].iterrows():
        conteudo_str=conteudo_str+ f'{texto[1][0]}: {texto[1][1]}'

conteudo_str

'\n==>> Aula 0: Teoria geral da empresa: <p>Estudante, esta videoaula foi preparada especialmente para você. Nela, você irá aprender conteúdos importantes para a sua formação profissional. Vamos assisti-la?</p><p>Bons estudos!</p>Ponto de Partida: <p>Olá, estudante! É um prazer poder iniciar mais uma jornada de aprendizado com você!</p><p>Na aula de hoje, exploraremos temas essenciais que moldam o panorama profissional contemporâneo. Para tanto, nossa abordagem apresentará uma visão abrangente dos conteúdos, destacando sua importância no desenvolvimento de habilidades cruciais para o mercado de trabalho atual. Diante disso, mergulharemos em discussões fundamentais sobre o empresário e a atividade empresarial, fundamentais para compreender o cenário jurídico dos negócios, o que não só ampliará seu repertório acadêmico, mas também o equipará com ferramentas práticas para enfrentar desafios do mundo real.</p><p>Ao longo da aula, preste atenção especial às distinções entre empresário indiv

In [26]:
len(conteudo_str)

397254

In [27]:
with open('conteudo_str', 'w') as arquivo:
    arquivo.write(conteudo_str)

----
<h3>Testes com LLMs</h3>

In [28]:
comando = questoes_prova.iloc[0][3]+questoes_prova.iloc[0][4]
resposta_prof = questoes_prova.iloc[0][5]
alternativas = ''
for n,i in enumerate(questoes_prova.iloc[0][6]):
    alternativas = alternativas + f"Alternativa {n}: {i['text']}. \n "

print( comando, alternativas, resposta_prof)

<p>A relação jurídica de consumo envolve dois sujeitos fundamentais: o consumidor e o fornecedor. Ambos desempenham papéis distintos e têm direitos e deveres definidos pelo Código de Defesa do Consumidor (CDC).</p><p>Assinale a alternativa que apresenta a característica marcante do consumidor na relação jurídica de consumo:&nbsp;</p> Alternativa 0: <p>Vulnerabilidade técnica.</p>. 
 Alternativa 1: <p>Conhecimento específico.</p>. 
 Alternativa 2: <p>Capacidade econômica superior.</p>. 
 Alternativa 3: <p>Participação no processo produtivo.</p>. 
 Alternativa 4: <p>Integração do produto em seu processo comercial.</p>. 
  <p>A característica marcante do consumidor na relação jurídica de consumo é a sua vulnerabilidade técnica, decorrente da impossibilidade quanto ao conhecimento específico de produtos ou serviços.</p>


In [29]:
#CHECKPOINT DE MEMORIA
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

with open('conteudo_str', 'r') as arquivo:
    conteudo_str = arquivo.read()
len(conteudo_str)

397254

In [30]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint='https://openai-dados-lab-poc.openai.azure.com/',
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)   

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [31]:
from uuid import uuid4
from langchain_core.documents import Document

documents = []
for i in conteudo_str.split('\n==>> '):
    documents.append(Document(i))
    
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['7067fa7f-b96c-451e-8f50-e784fc552a5e',
 '99aea7ac-3c7c-4de5-807f-3f69826e02e5',
 '2ea04521-896d-434e-8063-3fb76829e9e6',
 '9811b76a-d3e0-4644-9ead-06435f5301c9',
 'b3150300-c993-482e-a88f-2f72d8ec9e5f',
 '1ab35518-f764-48a2-8a5f-164e56e6e12d',
 '5a343db5-ea0f-47ee-9d43-0f8ca644421e',
 'b00830f4-b9c1-4397-a616-e0061c19d181',
 '3deaa3e0-893a-4608-920c-f3b6c1a3051e',
 '85e9db88-b34a-404d-891e-628a2922420c',
 '002bd0e2-d421-400b-95de-f3ddcd063946',
 'ee5849b3-b0ef-45c8-8c0c-88d8165380a4',
 'daa31f02-c2c9-4ba8-ae1a-2cfbaebe69e0',
 '5b6f83af-3c0e-437c-a1d3-330f244795f9',
 '09d4ee72-1309-4d08-94a6-fc3ade6bfd19',
 '91911293-7d6f-46d5-b86f-fbe201480e14',
 'b0d4f0b8-acdd-42e2-bb01-ea37588cf032',
 '5e468940-6e13-400c-97e9-4ae08cf8f464',
 'c1237af2-07ea-47ce-8019-6d12e45e5a65',
 '6c9e1b1d-306c-49d5-9b15-2de7c2eab5f6',
 'd1047018-e203-437e-8361-213e19f9e0b8']

In [39]:
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="https://openai-dados-lab-poc.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment="gpt-4o",
    openai_api_version="2024-02-01"
)

C:\Users\renan.vital\AppData\Local\Temp\ipykernel_24492\3739503124.py:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


<h3>RAG</h3>

<h3>Teste Roteiro de estudos</h3>

In [36]:
print('QUESTÂO: ',comando)

results = vector_store.similarity_search(comando, k=3)
for res in results:
    print(f"Aula relacionada: {res.page_content}")
    
conteudo_relacionado = "".join([i.page_content for i in results])
conteudo_relacionado

QUESTÂO:  <p>A relação jurídica de consumo envolve dois sujeitos fundamentais: o consumidor e o fornecedor. Ambos desempenham papéis distintos e têm direitos e deveres definidos pelo Código de Defesa do Consumidor (CDC).</p><p>Assinale a alternativa que apresenta a característica marcante do consumidor na relação jurídica de consumo:&nbsp;</p>
Aula relacionada: Aula 12: Relação jurídica de consumo: <p>Estudante, esta videoaula foi preparada especialmente para você. Nela, você irá aprender conteúdos importantes para a sua formação profissional. Vamos assisti-la?&nbsp;<br>Bons estudos!</p>Ponto de Partida: <p>Olá, estudante!</p><p>Nesta aula, estudaremos os fundamentos da relação jurídica de consumo, explorando os conceitos essenciais e os direitos básicos que permeiam a interação entre consumidores e fornecedores. Nesse contexto, abordaremos os princípios norteadores do direito do consumidor, como a vulnerabilidade, a equidade, a informação, a boa-fé objetiva e a segurança. Além disso, 

'Aula 12: Relação jurídica de consumo: <p>Estudante, esta videoaula foi preparada especialmente para você. Nela, você irá aprender conteúdos importantes para a sua formação profissional. Vamos assisti-la?&nbsp;<br>Bons estudos!</p>Ponto de Partida: <p>Olá, estudante!</p><p>Nesta aula, estudaremos os fundamentos da relação jurídica de consumo, explorando os conceitos essenciais e os direitos básicos que permeiam a interação entre consumidores e fornecedores. Nesse contexto, abordaremos os princípios norteadores do direito do consumidor, como a vulnerabilidade, a equidade, a informação, a boa-fé objetiva e a segurança. Além disso, examinaremos a legislação que rege esse campo, destacando o Código de Defesa do Consumidor (CDC) como uma fonte fundamental para a proteção e a defesa dos direitos dos consumidores. A compreensão desses princípios e dessas normas é crucial para garantir a justiça contratual, a transparência nas relações de consumo e a promoção da equidade entre as partes envolv

In [41]:
template_inicial = \
f"""System: Atue como um professor universitário que precisa explicar ao seu aluno
como resolver uma questão universitária de {materia} - {filtro_materia}.
Questão:  {comando}: {alternativas}.
Resposta correta: {resposta_prof}.
Apostila: {conteudo_relacionado}.
----------------------------------
Action: Primeiro explique o tema principal da questão. 
Depois utilize a Apostila que possui 3 aulas para explicar ao seu aluno
como estudar o assunto para compreender a questão, fazendo um roteiro de estudos.
Utilize no número da aula para explicar ao aluno onde localizar cada aula."""

completion_inicial = llm.invoke(template_inicial)
completion_inicial.content

'Olá, estudante!\n\nA questão que você apresentou aborda a relação jurídica de consumo, especificamente focando nas características do consumidor conforme definidas pelo Código de Defesa do Consumidor (CDC). Vamos começar explicando o tema principal da questão e, em seguida, fornecer um roteiro de estudos para que você possa compreender e aprofundar seu conhecimento sobre o assunto.\n\n### Tema Principal da Questão\n\nA relação jurídica de consumo envolve dois sujeitos fundamentais: o **consumidor** e o **fornecedor**. O consumidor é definido pelo CDC como a parte vulnerável dessa relação, pois geralmente não possui o mesmo nível de conhecimento técnico, econômico ou jurídico que o fornecedor. A questão exige que você identifique a característica marcante do consumidor, que é a **vulnerabilidade técnica**, decorrente da impossibilidade de possuir conhecimento específico sobre produtos ou serviços.\n\n### Roteiro de Estudos\n\nPara estudar e compreender melhor a questão, sugiro que você

'Olá, estudante!\n\nA questão que você apresentou aborda a relação jurídica de consumo, especificamente focando nas características do consumidor conforme definidas pelo Código de Defesa do Consumidor (CDC).  

 Vamos começar explicando o tema principal da questão e, em seguida, fornecer um roteiro de estudos para que você possa compreender e aprofundar seu conhecimento sobre o assunto.  
 
 ### Tema Principal da Questão  
 A relação jurídica de consumo envolve dois sujeitos fundamentais: o **consumidor** e o **fornecedor**. O consumidor é definido pelo CDC como a parte vulnerável dessa relação, pois geralmente não possui o mesmo nível de conhecimento técnico, econômico ou jurídico que o fornecedor. A questão exige que você identifique a característica marcante do consumidor, que é a **vulnerabilidade técnica**, decorrente da impossibilidade de possuir conhecimento específico sobre produtos ou serviços.
 
 ### Roteiro de Estudos  
 Para estudar e compreender melhor a questão, sugiro que você siga o roteiro de estudos abaixo, utilizando as aulas da apostila:  
 #### 1. Aula 12: Relação Jurídica de Consumo  
 **Ponto de Partida**: Inicie a leitura sobre os fundamentos da relação jurídica de consumo.  
 **Direito do Consumidor e seus Princípios**: Foco no **princípio da vulnerabilidade** (Art. 4º, inciso I do CDC). Entenda as três espécies de vulnerabilidade: fática, técnica e científica.  
 **Código de Defesa do Consumidor (CDC)**: Estude a origem e a importância do CDC para a proteção dos consumidores.  
 **Relação Jurídica de Consumo**: Analise a definição de consumidor e fornecedor, e compreenda como a vulnerabilidade do consumidor é fundamental para a aplicação do CDC.
 
 #### 2. Aula 15: Responsabilidade Civil nas Relações de Consumo  
 **Ponto de Partida**: Introdução à responsabilidade civil no contexto das relações de consumo.  
 **Responsabilidade Civil e Dever de Indenizar**: Entenda os conceitos de responsabilidade subjetiva e objetiva, com foco na responsabilidade objetiva que prevalece nas relações de consumo e como isso se relaciona com a vulnerabilidade do consumidor.  
 **Responsabilidade Civil pelo Fato e pelo Vício do Produto ou Serviço**: Diferença entre responsabilidade por fato e vício do produto ou serviço.  
 **Excludentes de Responsabilidade Civil nas Relações de Consumo**: Como o fornecedor pode, em certos casos, excluir sua responsabilidade.\n\n#### 3. Aula 13: Direitos Básicos do Consumidor  
 **Ponto de Partida**: Introdução aos direitos básicos do consumidor conforme o Art. 6º do CDC.   
 **Direitos Básicos do Consumidor**: \n  -   
 **Direito à Proteção da Vida, à Segurança e à Saúde**: Relaciona-se com a vulnerabilidade do consumidor.   
 **Direito à Educação, à Liberdade de Escolha e à Informação Adequada**: A importância da informação para mitigar a vulnerabilidade técnica.**Direito à Proteção contra Publicidade Enganosa e Abusiva**: Proteção do consumidor contra práticas que exploram sua vulnerabilidade.\n  -
 **Direito à Facilitação da Defesa dos Direitos e Inversão do Ônus da Prova**: Como a inversão do ônus da prova ajuda a equilibrar a relação de consumo.\n\n  
 
 ### Conclusão  
 Estudando essas aulas, você ganhará uma compreensão aprofundada dos princípios que regem a relação jurídica de consumo, especialmente a vulnerabilidade do consumidor. Este conhecimento é essencial para responder questões sobre os direitos e deveres dos consumidores e fornecedores conforme o CDC.\n\nEspero que este roteiro de estudos seja útil para sua compreensão e sucesso na disciplina de Legislação Empresarial Aplicada. Bons estudos!'





<h3>Teste Feedback</h3>


In [43]:
template_inicial = \
f"""System: Atue como um professor universitário que precisa explicar ao seu aluno
como resolver uma questão universitária de {materia} - {filtro_materia}.
Question:  {comando}: {alternativas}.
Right answer: {resposta_prof}.
Student answer: "Alternativa 1: <p>Conhecimento específico.</p>."
Conteúdo acadêmico: {conteudo_relacionado}
----------------------------------
Action: Explique se o aluno errou ou acertou a questão. Posteriormente utilize o conteúdo acadêmico para
explicar a resolução da questão utilizando um passo a passo."""

completion_inicial = llm.invoke(template_inicial)
completion_inicial.content

print(f"Questão de prova: {comando}: {alternativas}.")
print(f"Resposta: {resposta_prof}.")

Questão de prova: <p>A relação jurídica de consumo envolve dois sujeitos fundamentais: o consumidor e o fornecedor. Ambos desempenham papéis distintos e têm direitos e deveres definidos pelo Código de Defesa do Consumidor (CDC).</p><p>Assinale a alternativa que apresenta a característica marcante do consumidor na relação jurídica de consumo:&nbsp;</p>: Alternativa 0: <p>Vulnerabilidade técnica.</p>. 
 Alternativa 1: <p>Conhecimento específico.</p>. 
 Alternativa 2: <p>Capacidade econômica superior.</p>. 
 Alternativa 3: <p>Participação no processo produtivo.</p>. 
 Alternativa 4: <p>Integração do produto em seu processo comercial.</p>. 
 .
Resposta: <p>A característica marcante do consumidor na relação jurídica de consumo é a sua vulnerabilidade técnica, decorrente da impossibilidade quanto ao conhecimento específico de produtos ou serviços.</p>.


'Olá, estudante! Vamos analisar a sua resposta à questão e entender o motivo pelo qual você escolheu uma alternativa diferente da correta.

### Análise da Resposta  
Você escolheu a **Alternativa 1: Conhecimento específico**. No entanto, a correta é a **Alternativa 0: Vulnerabilidade técnica**.\n\n

### Explicação Passo a Passo  
#### 1. **Identificação dos Sujeitos na Relação de Consumo**
**Consumidor:** Toda pessoa física ou jurídica que adquire ou utiliza produto ou serviço como destinatário final.  
**Fornecedor:** Toda pessoa física ou jurídica, pública ou privada, nacional ou estrangeira, bem como os entes despersonalizados, que desenvolvem atividade de produção, montagem, criação, construção, transformação, importação, exportação, distribuição ou comercialização de produtos ou prestação de serviços.\n\n
#### 2. **Princípio da Vulnerabilidade**
O consumidor é considerado a parte vulnerável na relação de consumo. Essa vulnerabilidade pode ser categorizada em três tipos principais:\n     -
**Vulnerabilidade fática:**
 Relacionada à disparidade econômica e social entre o consumidor e o fornecedor.
 **Vulnerabilidade técnica:** 
 Decorrente da impossibilidade do consumidor quanto ao conhecimento específico de produtos ou serviços.\n
 **Vulnerabilidade científica:** 
 Relacionada à falta de informações sobre seus direitos e a ausência de assistência jurídica.\n\n
 
 #### 3. **Característica Marcante do Consumidor**
 De acordo com o **art. 4º, inciso I do Código de Defesa do Consumidor (CDC)**, a vulnerabilidade do consumidor é um princípio fundamental. Isso inclui a **vulnerabilidade técnica**, que é a incapacidade do consumidor de ter um conhecimento específico sobre os produtos ou serviços que consome.\n\n
 
 #### 4. **Análise das Alternativas**
 **Alternativa 0: Vulnerabilidade técnica.** (Correta) Esta alternativa está correta porque reflete a verdadeira característica do consumidor como a parte mais frágil na relação de consumo devido à sua falta de conhecimento técnico.\n  
 **Alternativa 1: Conhecimento específico.** (Incorreta)\n     - Esta alternativa está incorreta porque sugere que o consumidor possui conhecimento específico sobre os produtos ou serviços, o que contradiz a ideia de vulnerabilidade técnica.\n   -   
 **Alternativa 2: Capacidade econômica superior.** (Incorreta)\n     - Esta alternativa está incorreta porque, na maioria dos casos, o consumidor não possui capacidade econômica superior ao fornecedor.\n   -   
 **Alternativa 3: Participação no processo produtivo.** (Incorreta)\n     - Esta alternativa está incorreta porque o consumidor não participa do processo produtivo; ele é o destinatário final.\n   -   
 **Alternativa 4: Integração do produto em seu processo comercial.** (Incorreta)\n     - Esta alternativa está incorreta porque o consumidor usa o produto para seu próprio consumo e não para integrá-lo em um processo comercial.\n\n### Conclusão\n\nVocê escolheu a alternativa errada porque ela não reflete a vulnerabilidade técnica do consumidor na relação de consumo. A característica marcante do consumidor é a sua vulnerabilidade técnica, que decorre da falta de conhecimento específico sobre os produtos ou serviços que consome.\n\nEstude os princípios do direito do consumidor, especialmente a vulnerabilidade, para entender melhor as relações de consumo e a proteção que o Código de Defesa do Consumidor oferece.\n\nSe precisar de mais alguma coisa, estou à disposição! Bons estudos!'